In [1]:
import subprocess

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


/home/marumarukun/pj/my_study/whisper_demo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
YOUTUBE_ID = "Yr5ilUVvp4g"  # https://www.youtube.com/watch?v=wxvxP8yHEPg の末尾のID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"


def dl_yt(yt_url):
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True, check=False)


dl_yt(f"https://youtu.be/{YOUTUBE_ID}")


[youtube] Extracting URL: https://youtu.be/Yr5ilUVvp4g
[youtube] Yr5ilUVvp4g: Downloading webpage
[youtube] Yr5ilUVvp4g: Downloading ios player API JSON
[youtube] Yr5ilUVvp4g: Downloading web creator player API JSON
[youtube] Yr5ilUVvp4g: Downloading m3u8 information
[info] Yr5ilUVvp4g: Downloading 1 format(s): 251
[download] Yr5ilUVvp4g.mp3 has already been downloaded
[ExtractAudio] Not converting audio Yr5ilUVvp4g.mp3; file is already in target format mp3


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


In [4]:
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)


In [5]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


In [6]:
%%time

result = pipe(AUDIO_FILE_NAME, return_timestamps=True)


/home/marumarukun/pj/my_study/whisper_demo/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


CPU times: user 3min 14s, sys: 23.6 s, total: 3min 38s
Wall time: 3min 22s


In [7]:
for chunk in result["chunks"]:
    print(f"[{chunk['timestamp'][0]:.2f}s - {chunk['timestamp'][1]:.2f}s] {chunk['text']}")


[0.00s - 6.48s] いつもお聞きいただきありがとうございます
[6.48s - 10.72s] 今日はこれまで朗読した中から
[10.72s - 14.28s] 芥川龍之介の作品を集めてお届けします
[14.28s - 16.24s] どうぞお楽しみください
[0.00s - 1.70s] 蜘蛛の糸
[1.70s - 12.78s] ある日のことでございます
[12.78s - 19.48s] お釈迦様は極楽の蓮池の淵を
[19.48s - 23.02s] 一人でぶらぶらお歩きになっていらっしゃいました
[26.26s - 29.22s] 池の中に咲いている蓮の花は
[0.00s - 3.24s] みんな玉のように真っ白で
[3.24s - 8.26s] その真ん中にある金色の髄からは
[8.26s - 11.58s] なんとも言えない良い匂いが
[11.58s - 15.04s] 絶え間なく辺りへあふれております
[15.04s - 21.68s] 極楽はちょうど朝なのでございましょう
[0.00s - 9.82s] やがてお釈迦様はその池の淵にお佇みになって
[9.82s - 15.28s] 水の表を覆っている蓮の葉の間から
[15.28s - 18.68s] ふと下の様子をご覧になりました
[21.74s - 24.36s] この極楽の蓮池の下は
[24.36s - 27.84s] ちょうど地獄の底にあたっておりますから
[0.00s - 4.18s] 水晶のような水をすき通して
[4.18s - 8.98s] 山頭の川や梁の山の景色が
[8.98s - 12.44s] ちょうど覗き眼鏡を見るように
[12.44s - 15.08s] はっきりと見えるのでございます
[15.08s - 21.02s] するとその地獄の底に
[21.02s - 23.88s] 神田田という男が一人
[23.88s - 28.26s] 他の罪人と一緒にうごめいている姿が
[28.26s - 29.60s] お目にとまりました
[0.00s - 10.30s] この神田田という男は人を殺したり家に火をつけたり
[10.30s - 14.28s] いろいろ悪事を働いた大泥棒でございますが
[14.28s - 20.38s] それでもたった一つ良いことをいたした覚えがござい